In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from model import *
from test import test
import string
from utils import *
torch.manual_seed(1337)

In [3]:

def load_data(X,Y,block_size):
    features = []
    target = []
    def s_e(text,i,block_size):
        start = i
        end = i + block_size
        start = min(len(text) - block_size,start)
        end = min(len(text),end)
        return text[start:end]
    for f,t in zip(X,Y):
        for i in range(len(f) - block_size + 1):
            features.append(s_e(f,i,block_size))
            target.append(s_e(t,i,block_size))
    features = torch.tensor(features)
    target = torch.tensor(target)
    return features,target

In [4]:
N = 1
n_embeddings = 32
block_size = 10
n_head = 4

In [84]:
def prepare_data(
        src_path,
        dst_path,
        block_size,
        start_token="$",
        end_token = "^",
        add_start_end_tokens=True,
        device="cpu"
    ):
    src = load_txt(src_path)
    dst = load_txt(dst_path)

    src_vocab = get_vocab(src) if not add_start_end_tokens else get_vocab(src,start=start_token,end=end_token)
    dst_vocab = get_vocab(dst) if not add_start_end_tokens else get_vocab(dst,start=start_token,end=end_token)

    src = [encode(txt,src_vocab) for txt in src]
    dst = [encode(txt,dst_vocab) for txt in dst]
    
    src,dst = load_data(src,dst,block_size)
    if add_start_end_tokens:
        src = include_tokens(
            src,
            start_token_index=src_vocab.index(start_token),
        )
        dst = include_tokens(
            dst,
            end_token_index=dst_vocab.index(end_token),
        )
    return src.to(device=device),src_vocab,dst.to(device=device),dst_vocab
src,src_vocab,dst,dst_vocab = prepare_data(
    "./eng.txt",
    "./fr.txt",
    block_size,
    add_start_end_tokens=True,
)
encoder_vocab_size = len(src_vocab)
decoder_vocab_size = len(dst_vocab)

RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, ort, xla, lazy, vulkan, mps, meta, hpu, mtia, privateuseone device type at start of device string: gpu

In [86]:
device = "cpu"
model = AttentiveTranslator(
    1,
    encoder_vocab_size,
    decoder_vocab_size,
    n_embeddings,
    n_head,
    src.shape,
    dst.shape,
    device=device
).to(device=device)

In [7]:
from train import * 

In [18]:
optim = torch.optim.Adam(
    model.parameters()
)
loss_fn = torch.nn.functional.cross_entropy
data_loader = arrays_to_loader(src,dst,32)
train(model,optim,loss_fn,5,data_loader)

tensor(3.3406, grad_fn=<NllLossBackward0>)
tensor(3.2662, grad_fn=<NllLossBackward0>)
tensor(3.2202, grad_fn=<NllLossBackward0>)
tensor(3.1525, grad_fn=<NllLossBackward0>)
tensor(3.1225, grad_fn=<NllLossBackward0>)
tensor(3.0972, grad_fn=<NllLossBackward0>)
tensor(3.0068, grad_fn=<NllLossBackward0>)
tensor(2.9449, grad_fn=<NllLossBackward0>)
tensor(2.9824, grad_fn=<NllLossBackward0>)
tensor(2.8879, grad_fn=<NllLossBackward0>)
tensor(2.7828, grad_fn=<NllLossBackward0>)
tensor(2.7834, grad_fn=<NllLossBackward0>)
tensor(2.7198, grad_fn=<NllLossBackward0>)
tensor(2.6634, grad_fn=<NllLossBackward0>)
tensor(2.6538, grad_fn=<NllLossBackward0>)


In [73]:
len(src_vocab),len(dst_vocab)

(23, 28)

In [79]:
from test import test
test(model,"hello there ho are ou doing",block_size,src_vocab,dst_vocab)

'ppppppppppppppppp'